In [1]:
pip install -U vnstock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=18b9271d5158e2be389b1c962f00c75f6e66d3870f9e83db306b35a01087de32
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=e1ad6afd95525888b02429a7d2afc462e2fc236c3d34b1f802b2e24e119d5679
  Stored in directory: /root/.cache/pip/wheels/46/f9/96/709295c836133071c12a300729fed4027757f889c01695feea
Successfully built keras-self-attention
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import optuna
import joblib
from sklearn.metrics import (
    f1_score, accuracy_score, classification_report, roc_auc_score,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import TimeSeriesSplit

2025-06-20 14:30:27.289163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750429827.525252      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750429827.594637      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [9]:
def dataframe_to_x_y(df, feature, target):
  X_train, y_train = [], []
  X_val, y_val = [], []
  n = len(df)
  split_index = int(n * 0.8)
  X_train = df[feature][:split_index]
  y_train = df[target][:split_index]
  X_val = df[feature][split_index:]
  y_val = df[target][split_index:]
  return X_train, y_train, X_val, y_val

In [10]:

sentiment_columns = [
        'Reputation', 'Company Communication', 'Appointment', 'Financial', 'Regulatory',
        'Sales', 'M&A', 'Legal', 'Dividend Policy', 'Risks', 'Rumors', 'Strategy',
        'Options', 'IPO', 'Signal', 'Coverage', 'Fundamentals', 'Insider Activity',
        'Price Action', 'Buyside', 'Technical Analysis', 'Trade', 'Central Banks',
        'Currency', 'Conditions', 'Market', 'Volatility', 'Investor Sentiment', 'Retail Investor Behavior',
        'Speculation', 'Domestic Institutional Behavior', 'Foreign Institutional Behavior', 'Black Swan Event',
    ]

sentiment_p1d_columns = [
        'reputation_p1d', 'company_communication_p1d', 'appointment_p1d', 'financial_p1d', 'regulatory_p1d',
        'sales_p1d', 'm&a_p1d', 'legal_p1d', 'dividend_policy_p1d', 'risks_p1d', 'rumors_p1d', 'strategy_p1d',
        'options_p1d', 'ipo_p1d', 'signal_p1d', 'coverage_p1d', 'fundamentals_p1d', 'insider_activity_p1d',
        'price_action_p1d', 'buyside_p1d', 'technical_analysis_p1d', 'trade_p1d', 'central_banks_p1d',
        'currency_p1d', 'conditions_p1d', 'market_p1d', 'volatility_p1d', 'investor_sentiment_p1d', 'retail_investor_behavior_p1d',
        'speculation_p1d', 'domestic_institutional_behavior_p1d', 'foreign_institutional_behavior_p1d', 'black_swan_event_p1d',
    ]
meta_cols = ['num_sa_news', 'mean_sentiment_norm', 'max_sentiment_norm', 'std_sentiment_score', 'sum_positive_sa', 'sum_negative_sa', 'time_distance_from_sa_to_close_minutes']
meta_p1d_cols = ['num_sa_news_p1d', 'mean_sentiment_norm_p1d', 'max_sentiment_norm_p1d', 'std_sentiment_score_p1d', 'sum_positive_sa_p1d', 'sum_negative_sa_p1d', 'time_distance_from_sa_to_close_minutes_p1d']

sentiment_feature_selected =  sentiment_columns + sentiment_p1d_columns + meta_cols + meta_p1d_cols
ta_features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','roc_1','roc_9','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','rsi_base_ma','rsi_rsi_base_ma_ratio','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','hl_ratio', 'co_ratio', 'price_range', 'sma_ratio_20_50', 'ema_ratio_12_26', 'bb_width', 'bb_position', 'rsi_overbought', 'rsi_oversold', 'rsi_neutral', 'macd_bullish', 'momentum_5', 'momentum_10','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio',
               'rsi_vn30','rsi_base_ma_vn30','rsi_rsi_base_ma_ratio_vn30','volume_ma_vn30','volume_to_volume_ma_ratio_vn30','bb_bbm_vn30','bb_bbh_vn30','bb_bbl_vn30','bb_bbp_vn30','bb_bbh_bb_bbl_ratio_vn30','roc_1_vn30', 'roc_5_vn30', 'roc_9_vn30','%K_vn30','%R_vn30','cci_vn30','obv_vn30','ema_12_vn30','ema_26_vn30','sma_20_vn30','sma_50_vn30', 'hl_ratio_vn30', 'co_ratio_vn30', 'price_range_vn30', 'sma_ratio_20_50_vn30', 'ema_ratio_12_26_vn30', 'bb_width_vn30', 'bb_position_vn30', 'rsi_overbought_vn30', 'rsi_oversold_vn30', 'rsi_neutral_vn30', 'momentum_5_vn30', 'momentum_10_vn30', 'log_return_vn30','volatility_5d_vn30','volatility_10d_vn30','volatility_20d_vn30','volatility_30d_vn30','mean_log_return_5d_vn30','mean_log_return_10d_vn30','mean_log_return_20d_vn30','mean_log_return_30d_vn30','sharpe_like_5d_vn30','sharpe_like_10d_vn30','sharpe_like_20d_vn30','sharpe_like_30d_vn30','up_streak_vn30','pos_log_return_ratio_20d_vn30','z_score_5d_vn30','z_score_10d_vn30','z_score_20d_vn30','z_score_30d_vn30','annual_return_vn30','daily_return_vn30','sharpe_ratio_vn30',
               'rsi_vni','rsi_base_ma_vni','rsi_rsi_base_ma_ratio_vni','volume_ma_vni','volume_to_volume_ma_ratio_vni','bb_bbm_vni','bb_bbh_vni','bb_bbl_vni','bb_bbp_vni','bb_bbh_bb_bbl_ratio_vni','roc_1_vni', 'roc_5_vni', 'roc_9_vni','%K_vni','%R_vni','cci_vni','obv_vni','ema_12_vni','ema_26_vni','sma_20_vni','sma_50_vni', 'hl_ratio_vni', 'co_ratio_vni', 'price_range_vni', 'sma_ratio_20_50_vni', 'ema_ratio_12_26_vni', 'bb_width_vni', 'bb_position_vni', 'rsi_overbought_vni', 'rsi_oversold_vni', 'rsi_neutral_vni', 'momentum_5_vni', 'momentum_10_vni','log_return_vni','volatility_5d_vni','volatility_10d_vni','volatility_20d_vni','volatility_30d_vni','mean_log_return_5d_vni','mean_log_return_10d_vni','mean_log_return_20d_vni','mean_log_return_30d_vni','sharpe_like_5d_vni','sharpe_like_10d_vni','sharpe_like_20d_vni','sharpe_like_30d_vni','up_streak_vni','pos_log_return_ratio_20d_vni','z_score_5d_vni','z_score_10d_vni','z_score_20d_vni','z_score_30d_vni','annual_return_vni','daily_return_vni','sharpe_ratio_vni']
fa_features = ['p/b_previous_quarter', 'p/b_change_rate','p/b_change_rate_flag','p/e_previous_quarter','p/e_change_rate','p/e_change_rate_flag','p/s_previous_quarter','p/s_change_rate','p/s_change_rate_flag','p/cash_flow_previous_quarter','p/cash_flow_change_rate','p/cash_flow_change_rate_flag','eps_previous_quarter','eps_change_rate', 'eps_change_rate_flag','bvps_previous_quarter','bvps_change_rate', 'bvps_change_rate_flag','roe_previous_quarter','roe_change_rate','roe_change_rate_flag','roa_previous_quarter','roa_change_rate','roa_change_rate_flag','coefficient_p/b','coefficient_p/e','coefficient_p/s','coefficient_p/cash_flow','coefficient_eps','coefficient_bvps','coefficient_roe','coefficient_roa','distance_to_nearest_quarter']
ta_fa_feature_selected = ta_features + fa_features
features = ta_fa_feature_selected + sentiment_feature_selected
target = 'target'
scaler = StandardScaler()

In [11]:
def load_data_for_analysis(tickers, folder_path):

    df_all = []

    for ticker in tickers:
        file_path = os.path.join(folder_path, f"{ticker}.xlsx")
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            df["ticker"] = ticker  # thêm cột ticker
            df_all.append(df)
        else:
            print(f"⚠️ Không tìm thấy: {file_path}")

    # Gộp lại thành 1 DataFrame
    df_global = pd.concat(df_all, ignore_index=True)
    print(f"✅ Đã load xong {len(df_global)} dòng dữ liệu từ {len(df_all)} cổ phiếu.")
    return df_global

In [12]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG',
                    'LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB',
                    'TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
folder_path_train = "/kaggle/input/vn30-dataset/v6 - Copy/out_performance_vn30/train_data" 
folder_path_test = "/kaggle/input/vn30-dataset/v6 - Copy/out_performance_vn30/test_data"
df_global_train = load_data_for_analysis(tickers, folder_path_train)
df_global_test = load_data_for_analysis(tickers, folder_path_test)


✅ Đã load xong 35865 dòng dữ liệu từ 30 cổ phiếu.
✅ Đã load xong 7380 dòng dữ liệu từ 30 cổ phiếu.


In [13]:
# Danh sách các mô hình
models = {
    "Random Forest": RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    "LightGBM": LGBMClassifier(n_jobs=-1, random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
}

In [14]:
X_train_no_sa = df_global_train[ta_fa_feature_selected]
X_test_no_sa = df_global_test[ta_fa_feature_selected]
y_train = df_global_train[target]
y_test = df_global_test[target]
scaler = StandardScaler()
X_train_no_sa_scaled = scaler.fit_transform(X_train_no_sa)
X_test_no_sa_scaled = scaler.transform(X_test_no_sa) 

In [15]:
X_train_with_sa = df_global_train[features]  # features = ta + fa + sa
X_test_with_sa = df_global_test[features]
y_train = df_global_train[target]
y_test = df_global_test[target]

scaler = StandardScaler()
X_train_with_sa_scaled = scaler.fit_transform(X_train_with_sa)   # Fit scaler trên train, scale train luôn
X_test_with_sa_scaled = scaler.transform(X_test_with_sa)  

In [16]:
# ==============================================Tuning with Optuna + TimeSeriesSplit validate (AUC macro) ===============================================

Divide 5 fold to cross validation though tuning parameters

In [17]:
def get_grouped_time_series_folds(df, ticker_col='ticker', n_splits=5):
    tickers = df[ticker_col].unique()
    fold_indices = [[] for _ in range(n_splits)]  # List chứa n_splits fold tổng
    for ticker in tickers:
        idx = df[df[ticker_col] == ticker].index.values
        tscv = TimeSeriesSplit(n_splits=n_splits)
        for fold_num, (train_idx, val_idx) in enumerate(tscv.split(idx)):
            # Gộp index thực tế vào fold tổng tương ứng
            fold_indices[fold_num].append((idx[train_idx], idx[val_idx]))
    # Gộp index của từng fold tổng lại thành mảng train/val duy nhất
    grouped_folds = []
    for fold in fold_indices:
        train_idx = np.concatenate([item[0] for item in fold])
        val_idx = np.concatenate([item[1] for item in fold])
        grouped_folds.append((train_idx, val_idx))
    return grouped_folds

In [18]:
folds = get_grouped_time_series_folds(df_global_train, ticker_col='ticker', n_splits=5)

Tuning random forest for stock trend with ta + fa data

In [19]:
X_no_sa = X_train_no_sa_scaled  # hoặc X_train_with_sa_scaled nếu dùng cả SA
y = y_train

In [20]:
print("Tuning LightGBM with Optuna (AUC macro) ta + fa features only")

def objective_ligbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1200),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.8),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "class_weight": "balanced",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "random_state": 42,
        "n_jobs": -1,
        "device": "gpu",           # <--- Thêm dòng này để dùng GPU
        # "gpu_platform_id": 0,    # <--- Tuỳ chọn
        # "gpu_device_id": 0,      # <--- Tuỳ chọn
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_no_sa[train_idx], X_no_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_ligbm, n_trials=50)

print("Best AUC macro (LGBMClassifier):", -study.best_value)
print("Best params (LGBMClassifier):", study.best_trial.params)

[I 2025-06-20 14:33:36,889] A new study created in memory with name: no-name-bcff3816-f011-4569-93e0-61dba2cb8784


Tuning LightGBM with Optuna (AUC macro) ta + fa features only


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
[I 2025-06-20 14:34:57,293] Trial 0 finished with value: -0.5755153083293036 and parameters: {'n_estimators': 805, 'max_depth': 6, 'learning_rate': 0.07137584452893188, 'num_leaves': 72, 'subsample': 0.9648828939740106, 'colsample_bytree': 0.5945359742528653, 'min_child_samples': 57}. Best is trial 0 with 

Trial 0: AUC macro=0.57552, params={'n_estimators': 805, 'max_depth': 6, 'learning_rate': 0.07137584452893188, 'num_leaves': 72, 'subsample': 0.9648828939740106, 'colsample_bytree': 0.5945359742528653, 'min_child_samples': 57}


[I 2025-06-20 14:35:34,198] Trial 1 finished with value: -0.5893823941101198 and parameters: {'n_estimators': 477, 'max_depth': 7, 'learning_rate': 0.06410653650840283, 'num_leaves': 21, 'subsample': 0.9169356086234672, 'colsample_bytree': 0.46379047502846216, 'min_child_samples': 58}. Best is trial 1 with value: -0.5893823941101198.


...

Trial 47: AUC macro=0.59214, params={'n_estimators': 670, 'max_depth': 7, 'learning_rate': 0.02443357717796909, 'num_leaves': 45, 'subsample': 0.9022599169208024, 'colsample_bytree': 0.42582736419483336, 'min_child_samples': 50}


[I 2025-06-20 16:03:31,699] Trial 48 finished with value: -0.5972403796168929 and parameters: {'n_estimators': 367, 'max_depth': 10, 'learning_rate': 0.037813520779085646, 'num_leaves': 16, 'subsample': 0.8791190276198527, 'colsample_bytree': 0.5695483983943374, 'min_child_samples': 62}. Best is trial 42 with value: -0.6039986555250194.


Trial 48: AUC macro=0.59724, params={'n_estimators': 367, 'max_depth': 10, 'learning_rate': 0.037813520779085646, 'num_leaves': 16, 'subsample': 0.8791190276198527, 'colsample_bytree': 0.5695483983943374, 'min_child_samples': 62}


[I 2025-06-20 16:04:47,233] Trial 49 finished with value: -0.6004874710217039 and parameters: {'n_estimators': 512, 'max_depth': 8, 'learning_rate': 0.01639806003011833, 'num_leaves': 84, 'subsample': 0.9171759904255407, 'colsample_bytree': 0.3367212456984284, 'min_child_samples': 56}. Best is trial 42 with value: -0.6039986555250194.


Trial 49: AUC macro=0.60049, params={'n_estimators': 512, 'max_depth': 8, 'learning_rate': 0.01639806003011833, 'num_leaves': 84, 'subsample': 0.9171759904255407, 'colsample_bytree': 0.3367212456984284, 'min_child_samples': 56}
Best AUC macro (LGBMClassifier): 0.6039986555250194
Best params (LGBMClassifier): {'n_estimators': 678, 'max_depth': 10, 'learning_rate': 0.010451129787383398, 'num_leaves': 41, 'subsample': 0.9070209365911411, 'colsample_bytree': 0.3253488510157635, 'min_child_samples': 69}


Tuning random forest for stock trend with ta + fa + sa data

In [21]:
X_with_sa = X_train_with_sa_scaled  
y = y_train

In [22]:
print("Tuning LightGBM with Optuna (AUC macro) ta + fa + sa features")

def objective_ligbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 400, 1400),
        "max_depth": trial.suggest_int("max_depth", 6, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.12),
        "num_leaves": trial.suggest_int("num_leaves", 32, 256),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 0.7),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 120),
        "class_weight": "balanced",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "random_state": 42,
        "n_jobs": -1,
        "device": "gpu",
        "verbosity": -1,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_with_sa[train_idx], X_with_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_ligbm, n_trials=50)

print("Best AUC macro (LGBMClassifier):", -study.best_value)
print("Best params (LGBMClassifier):", study.best_trial.params)

[I 2025-06-20 16:04:47,260] A new study created in memory with name: no-name-840546d8-8166-48c3-b993-2c87837f7e80


Tuning LightGBM with Optuna (AUC macro) ta + fa + sa features


[I 2025-06-20 16:07:58,192] Trial 0 finished with value: -0.5942156464871686 and parameters: {'n_estimators': 974, 'max_depth': 15, 'learning_rate': 0.10803244687596046, 'num_leaves': 172, 'subsample': 0.7100219629482889, 'colsample_bytree': 0.25134603712920023, 'min_child_samples': 108}. Best is trial 0 with value: -0.5942156464871686.


Trial 0: AUC macro=0.59422, params={'n_estimators': 974, 'max_depth': 15, 'learning_rate': 0.10803244687596046, 'num_leaves': 172, 'subsample': 0.7100219629482889, 'colsample_bytree': 0.25134603712920023, 'min_child_samples': 108}


[I 2025-06-20 16:15:04,815] Trial 1 finished with value: -0.5954077829664324 and parameters: {'n_estimators': 1279, 'max_depth': 12, 'learning_rate': 0.07072464808241466, 'num_leaves': 104, 'subsample': 0.7677646537896415, 'colsample_bytree': 0.4264965038200422, 'min_child_samples': 27}. Best is trial 1 with value: -0.5954077829664324.


...

Trial 47: AUC macro=0.60899, params={'n_estimators': 525, 'max_depth': 16, 'learning_rate': 0.027206092529229835, 'num_leaves': 228, 'subsample': 0.8805238796781735, 'colsample_bytree': 0.28714968123769224, 'min_child_samples': 106}


[I 2025-06-20 18:33:12,672] Trial 48 finished with value: -0.6123941047885778 and parameters: {'n_estimators': 580, 'max_depth': 13, 'learning_rate': 0.014600986230816099, 'num_leaves': 75, 'subsample': 0.8094733286145381, 'colsample_bytree': 0.5587981661587806, 'min_child_samples': 94}. Best is trial 24 with value: -0.6178919025987575.


Trial 48: AUC macro=0.61239, params={'n_estimators': 580, 'max_depth': 13, 'learning_rate': 0.014600986230816099, 'num_leaves': 75, 'subsample': 0.8094733286145381, 'colsample_bytree': 0.5587981661587806, 'min_child_samples': 94}


[I 2025-06-20 18:35:52,177] Trial 49 finished with value: -0.5990979725481147 and parameters: {'n_estimators': 786, 'max_depth': 14, 'learning_rate': 0.05128520892002661, 'num_leaves': 210, 'subsample': 0.8940758866130323, 'colsample_bytree': 0.3826126978169565, 'min_child_samples': 116}. Best is trial 24 with value: -0.6178919025987575.


Trial 49: AUC macro=0.59910, params={'n_estimators': 786, 'max_depth': 14, 'learning_rate': 0.05128520892002661, 'num_leaves': 210, 'subsample': 0.8940758866130323, 'colsample_bytree': 0.3826126978169565, 'min_child_samples': 116}
Best AUC macro (LGBMClassifier): 0.6178919025987575
Best params (LGBMClassifier): {'n_estimators': 488, 'max_depth': 16, 'learning_rate': 0.010304172272245919, 'num_leaves': 135, 'subsample': 0.9539484209180064, 'colsample_bytree': 0.20347358093479598, 'min_child_samples': 97}
